In [1]:
from src.fforma import FFORMA
from sklearn.decomposition import PCA
from src.meta_learner import MetaLearnerNN
from src.meta_results_r_data import prepare_fforma_data

from ESRNN.m4_data import prepare_m4_data, seas_dict
from ESRNN.utils_evaluation import evaluate_prediction_owa

from src.benchmarks import FactorQuantileRegressionAveraging

import pandas as pd
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pickle

from statsmodels.regression.quantile_regression import QuantReg

from src.metrics.metrics import smape, mape
from src.benchmarks import evaluate_panel

from src.base_models import FQRA, QRAL1

/Users/kin_gutierrez/opt/anaconda3/envs/fforma/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/kin_gutierrez/opt/anaconda3/envs/fforma/lib/python3.6/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = prepare_fforma_data(directory='data', dataset_name=None, kind='TOURISM')
#dataset_name=None for all data

% of missing values, train set
x_acf10               7.551487
diff1_acf1            7.551487
diff1_acf10           7.551487
diff2_acf1            7.551487
diff2_acf10           8.543097
seas_acf1            39.511823
arch_lm               9.687262
garch_acf             0.076278
arch_r2              13.196034
garch_r2             13.348589
nonlinearity          1.144165
x_pacf5               1.144165
diff1x_pacf5          1.830664
diff2x_pacf5          2.212052
seas_pacf            39.511823
e_acf10               7.551487
seasonal_strength    39.511823
peak                 39.511823
trough               39.511823
hw_alpha             39.511823
hw_beta              39.511823
hw_gamma             39.511823
dtype: float64


% of missing values, test set
x_acf10               1.830664
diff1_acf1            1.830664
diff1_acf10           2.212052
diff2_acf1            2.212052
diff2_acf10           7.551487
seas_acf1            39.511823
arch_lm               7.551487
arch_r2               7

In [3]:
X_train = data['preds_train_df']
y_train = data['y_train_df']
X_test = data['preds_test_df']
y_test = data['y_test_df']

In [4]:
X_train.dtypes

unique_id            object
ds                    int64
auto_arima_forec    float64
ets_forec           float64
naive_forec         float64
nnetar_forec        float64
rw_drift_forec      float64
snaive_forec        float64
stlm_ar_forec       float64
tbats_forec         float64
theta_forec         float64
dtype: object

# PCA for residuals

In [5]:
unique_ids = X_train['unique_id'].unique()

X_train_pca = X_train.drop(columns=['unique_id','ds'])#.values
X_test_pca = X_train.drop(columns=['unique_id','ds'])#.values

def demean_x(X_df):
    x_mean_list = []
    x_demean_list = []
    unique_ids = X_df['unique_id'].unique()
    for u_id in unique_ids:
        x_id = X_df[X_df['unique_id']==u_id].drop(columns=['unique_id','ds']).values
        x_mean = np.expand_dims(np.mean(x_id, axis=1), axis=1)
        x_id_demean = x_id - x_mean
        x_demean_list.append(x_id_demean)
        x_mean_list.append(x_mean)
    
    x_mean = np.vstack(x_mean_list)
    x_demean = np.vstack(x_demean_list)
    return x_mean, x_demean

X_train_mean, X_train_pca = demean_x(X_train)
X_test_mean, X_test_pca = demean_x(X_test)

print("X_train_mean.shape", X_train_mean.shape)
print("X_train_pca.shape", X_train_pca.shape)
print("X_test_mean.shape", X_test_mean.shape)
print("X_test_pca.shape", X_test_pca.shape)

pca_model = PCA(n_components=1).fit(X_train_pca)
X_train_transformed = pca_model.transform(X_train_pca)
X_test_transformed = pca_model.transform(X_test_pca)

X_train_mean.shape (14272, 1)
X_train_pca.shape (14272, 9)
X_test_mean.shape (14272, 1)
X_test_pca.shape (14272, 9)


# Quantile Averaging

In [6]:
smape_list = []
y_hat_list = []
u_id_list = []
ds_list = []
for u_id in unique_ids:
    x_train_transformed_id = X_train_transformed[(X_train['unique_id']==u_id)]
    x_train_mean_id = X_train_mean[(X_train['unique_id']==u_id)]
    
    x_test_transformed_id = X_test_transformed[(X_test['unique_id']==u_id)]
    x_test_mean_id = X_test_mean[(X_test['unique_id']==u_id)]
    
    y_train_id = y_train[y_train['unique_id']==u_id]['y'].values
    y_test_id = y_test[y_test['unique_id']==u_id]['y'].values

#     X_tr = np.hstack([np.ones(shape=(len(x_train_mean_id), 1)),
#                      x_train_transformed_id, x_train_mean_id])
    X_tr = np.hstack([x_train_transformed_id, x_train_mean_id])
    reg = QRAL1(tau=0.4, lambd=0.2).fit(X_tr, y_train_id)

#     X_ts = np.hstack([np.ones(shape=(len(x_train_mean_id), 1)),
#                      x_test_transformed_id, x_test_mean_id])
    X_ts = np.hstack([x_test_transformed_id, x_test_mean_id])
    y_hat = reg.predict(X_ts)

    my_smape=200*np.mean(np.abs(y_hat-y_test_id)/(np.abs(y_hat)+np.abs(y_test_id)))
    smape_list.append(my_smape)
    y_hat_list += y_hat.tolist()
    u_id_list += [u_id]*len(y_hat)
    ds_list += list(range(len(y_hat)))
    
np.mean(smape_list)

25.542398384043352